In [ ]:
import torch
import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.models import RNNModel
from darts.metrics import mae, mse, rmse
from darts.dataprocessing.transformers import Scaler
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Cargar y preparar datos
df = pd.read_csv("processed_dataset.csv", index_col="Date")
df.index = pd.to_datetime(df.index)

# Crear features temporales
df["Month_sin"] = np.sin(2 * np.pi * df.index.month / 12)
df["Month_cos"] = np.cos(2 * np.pi * df.index.month / 12)
df["Year"] = df.index.year

# Seleccionar features
features = ["SIN", "Temperature", "Humidity", "Month_sin", "Month_cos", "Year"]

# Crear series de tiempo con Darts
series = TimeSeries.from_dataframe(df[features], freq='H')

# Separar en train y validation
train_series = series[:-26304]  # 2009-2019 (3 años de validación: 2020-2022)
val_series = series[-26304:]

# Definir variables covariables y target
target_series = train_series["SIN"]
val_target = val_series["SIN"]

# Usar todas las features excepto SIN como covariables
covariates = train_series[["Temperature", "Humidity", "Month_sin", "Month_cos", "Year"]]
val_covariates = val_series[["Temperature", "Humidity", "Month_sin", "Month_cos", "Year"]]

# Escalar los datos
target_scaler = Scaler(MinMaxScaler())
covariates_scaler = Scaler(MinMaxScaler())

target_scaled = target_scaler.fit_transform(target_series)
val_target_scaled = target_scaler.transform(val_target)

covariates_scaled = covariates_scaler.fit_transform(covariates)
val_covariates_scaled = covariates_scaler.transform(val_covariates)

# Configuración de parámetros del modelo
model = RNNModel(
    model="GRU",
    input_chunk_length=336,        # hist_window
    output_chunk_length=24,        # pred_window
    hidden_dim=48,                 # num_units
    n_rnn_layers=1,
    dropout=0.2,                   # dropout
    batch_size=16,                 # batch_size
    n_epochs=150,                  # epochs
    optimizer_kwargs={"lr": 5e-4}, # learning_rate
    loss_fn="mae",                 # Mean Absolute Error
    random_state=7,                # SEED
    training_length=336,           # igual a input_chunk_length
    force_reset=True,
    save_checkpoints=True,
    pl_trainer_kwargs={
        "accelerator": "gpu" if torch.cuda.is_available() else "cpu",
        "devices": [0] if torch.cuda.is_available() else None
    }
)

# Entrenar el modelo
print("Entrenando modelo...")
model.fit(
    series=target_scaled,
    future_covariates=covariates_scaled,
    val_series=val_target_scaled,
    val_future_covariates=val_covariates_scaled,
    verbose=True
)

# Realizar predicciones
print("Realizando predicciones...")
# Predecir en el período de validación
predictions = model.historical_forecasts(
    series=val_target_scaled,
    future_covariates=val_covariates_scaled,
    start=val_target_scaled.start_time(),
    forecast_horizon=24,
    stride=24,
    retrain=False,
    verbose=True
)

# Revertir scaling de las predicciones
predictions_original = target_scaler.inverse_transform(predictions)
val_original = target_scaler.inverse_transform(val_target_scaled)

# Calcular métricas
mae_val = mae(val_original, predictions_original)
mse_val = mse(val_original, predictions_original)
rmse_val = rmse(val_original, predictions_original)

print(f"MAE: {mae_val:.4f}")
print(f"MSE: {mse_val:.4f}")
print(f"RMSE: {rmse_val:.4f}")

# Visualización de resultados
plt.figure(figsize=(12, 6))

# Tomar solo una parte para visualización clara
plot_start = pd.Timestamp('2020-01-01')
plot_end = pd.Timestamp('2020-02-01')

val_plot = val_original.drop_before(plot_start).drop_after(plot_end)
pred_plot = predictions_original.drop_before(plot_start).drop_after(plot_end)

val_plot.plot(label='Real', color='red', alpha=0.7)
pred_plot.plot(label='Predicho', alpha=0.7)

plt.title('Comparación: Valores Reales vs Predichos')
plt.xlabel('Fecha')
plt.ylabel('SIN (MW)')
plt.legend()
plt.grid(True)
plt.show()

# Visualización de todo el período de validación
plt.figure(figsize=(15, 6))
val_original.plot(label='Real', color='red', alpha=0.7)
predictions_original.plot(label='Predicho', alpha=0.7)
plt.title('Predicciones vs Valores Reales - Período Completo de Validación')
plt.xlabel('Fecha')
plt.ylabel('SIN (MW)')
plt.legend()
plt.grid(True)
plt.show()

# Guardar el modelo
model.save("modelo_darts.pkl")
print("Modelo guardado como 'modelo_darts.pkl'")

# Ejemplo de predicción futura
print("\nEjemplo de predicción futura:")
last_points = series[-336:]  # Últimos 336 puntos para predicción
last_covariates = covariates_scaler.transform(
    series[["Temperature", "Humidity", "Month_sin", "Month_cos", "Year"]]
)[-336:]

future_forecast = model.predict(
    n=24,
    series=target_scaled,
    future_covariates=covariates_scaled
)

future_forecast_original = target_scaler.inverse_transform(future_forecast)
print(f"Predicción para las próximas 24 horas: {future_forecast_original.values()}")

h:\Anaconda\envs\deepf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
h:\Anaconda\envs\deepf\lib\site-packages\darts\timeseries.py:5120: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  resampled_times = resampled_times.asfreq(freq)
ignoring user defined `output_chunk_length`. RNNModel uses a fixed `output_chunk_length=1`.
ValueError: The model does not support `past_covariates`. It only supports `future_covariates`.


Entrenando modelo...


ValueError: The model does not support `past_covariates`. It only supports `future_covariates`.

In [2]:
!pip install torch